In [42]:
#importing all the essential libraries 
import tensorflow as tf
import numpy as np
import numpy as np 
import pandas as pd 
import os, time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import keras
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16;
from keras.applications.vgg16 import preprocess_input
import os

In [43]:
#loading mnist dataset from keras
mnist = tf.keras.datasets.mnist

In [44]:
#trainX is an array, etc, arraying from 0-255
(trainX, trainY), (testX, testY) = mnist.load_data()
print(trainX.shape, testX.shape)

(60000, 28, 28) (10000, 28, 28)


In [45]:
trainX=np.dstack([trainX] * 3)
testX=np.dstack([testX]*3)
trainX.shape,testX.shape

((60000, 28, 84), (10000, 28, 84))

In [46]:
#reshape images as per the tensor format required by tensorflow
trainX = trainX.reshape(-1, 28,28,3)
testX= testX.reshape (-1,28,28,3)
trainX.shape,testX.shape

((60000, 28, 28, 3), (10000, 28, 28, 3))

In [48]:
#resize the images 48*48 as required by VGG16
from keras.preprocessing.image import img_to_array, array_to_img
trainX = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in trainX])
testX = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in testX])

#trainx = preprocess_input(x)
trainX.shape, testX.shape
classes = np.unique(trainY)
num_classes = len(classes)
num_classes

10

In [49]:
# Normalise the data and change data type
trainX = trainX / 255.
testX = testX / 255.
trainX = trainX.astype('float32')
testX = testX.astype('float32')

In [50]:
#converting Labels to one hot encoded format
trainY_one_hot = to_categorical(trainY)
testY_one_hot = to_categorical(testY)

trainX,valid_X,train_label,valid_label = train_test_split(trainX, trainY_one_hot,test_size=0.2,random_state=13)

In [51]:
#finally check the data size whether it is as per tensorflow and VGG16 requirement
trainX.shape,valid_X.shape,train_label.shape,valid_label.shape

#define the parameters for instanitaing VGG16 model. 
IMG_WIDTH = 48
IMG_HEIGHT = 48
IMG_DEPTH = 3
BATCH_SIZE = 16

#preprocessing the input 
trainX = preprocess_input(trainX)
valid_X = preprocess_input(valid_X)
testX  = preprocess_input (testX)

#create base model of VGG16
conv_base = VGG16(weights='imagenet',include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_DEPTH))
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0     

In [52]:
#extracting features
train_features = conv_base.predict(np.array(trainX), batch_size=BATCH_SIZE, verbose=1)
test_features = conv_base.predict(np.array(testX), batch_size=BATCH_SIZE, verbose=1)
val_features = conv_base.predict(np.array(valid_X), batch_size=BATCH_SIZE, verbose=1)

750/750 [==============================] - 181s 241ms/step


In [53]:
#saving the features so that they can be used for future
np.savez("train_features", train_features, train_label)
np.savez("test_features", test_features, testY)
np.savez("val_features", val_features, valid_label)

In [54]:
train_features_flat = np.reshape(train_features, (48000, 1*1*512))
test_features_flat = np.reshape(test_features, (10000, 1*1*512))
val_features_flat = np.reshape(val_features, (12000, 1*1*512))

(48000, 1, 1, 512) 
 (10000, 1, 1, 512) 
 (12000, 1, 1, 512)


In [55]:
from keras import models
from keras.models import Model
from keras import layers
from keras import optimizers
from keras import callbacks
from keras.layers.advanced_activations import LeakyReLU

In [56]:
NB_TRAIN_SAMPLES = train_features_flat.shape[0]
NB_VALIDATION_SAMPLES = val_features_flat.shape[0]
NB_EPOCHS = 100

model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_dim=(1*1*512)))
model.add(layers.LeakyReLU(alpha=0.1))
model.add(layers.Dense(num_classes, activation='softmax'))

In [57]:
model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(),metrics=['acc'])

In [58]:
reduce_learning = callbacks.ReduceLROnPlateau(monitor='val_loss',factor=0.2,patience=2,verbose=1,mode='auto',epsilon=0.0001,cooldown=2,min_lr=0)
eary_stopping = callbacks.EarlyStopping(monitor='val_loss',min_delta=0,patience=7,verbose=1,mode='auto')
callbacks = [reduce_learning, eary_stopping]

In [59]:
#train the model
history = model.fit(train_features_flat,train_label,epochs=NB_EPOCHS,validation_data=(val_features_flat, valid_label),callbacks=callbacks)

Epoch 1/100
1500/1500 [==============================] - 7s 5ms/step - loss: 1.4029 - acc: 0.5027 - val_loss: 1.1159 - val_acc: 0.6132
Epoch 2/100
1500/1500 [==============================] - 8s 5ms/step - loss: 0.9956 - acc: 0.6545 - val_loss: 0.9222 - val_acc: 0.6880
Epoch 3/100
1500/1500 [==============================] - 9s 6ms/step - loss: 0.8811 - acc: 0.6974 - val_loss: 0.8306 - val_acc: 0.7152
Epoch 4/100
1500/1500 [==============================] - 9s 6ms/step - loss: 0.8077 - acc: 0.7267 - val_loss: 0.7674 - val_acc: 0.7370
Epoch 5/100
1500/1500 [==============================] - 8s 5ms/step - loss: 0.7633 - acc: 0.7389 - val_loss: 0.6876 - val_acc: 0.7731
Epoch 6/100
1500/1500 [==============================] - 7s 5ms/step - loss: 0.7266 - acc: 0.7535 - val_loss: 0.7686 - val_acc: 0.7327
Epoch 7/100
1493/1500 [============================>.] - ETA: 0s - loss: 0.7064 - acc: 0.7609- E
Epoch 00007: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
1500/1500 [=

In [ ]:
S